In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from bsmcalls import individuals
import attila_utils
%matplotlib inline

In [2]:
calls = individuals.get_data(merge=True, cleancalls=False)
calls.columns

Index(['REF', 'ALT', 'QUAL', 'FILTER/PASS', 'FILTER/HC', 'FILTER/EXT', 'AC',
       'AF', 'AN', 'BaseQRankSum', 'DP', 'DS', 'END', 'ExcessHet', 'FS',
       'InbreedingCoeff', 'MLEAC', 'MLEAF', 'MQ', 'MQRankSum',
       'NEGATIVE_TRAIN_SITE', 'POSITIVE_TRAIN_SITE', 'QD', 'RAW_MQandDP',
       'ReadPosRankSum', 'SOR', 'VQSLOD', 'culprit', 'ChromatinState_DLPFC',
       'szdbCNVcount', 'GWASpval', 'Institution', 'Reported Gender',
       'Ethnicity', 'ageOfDeath', 'Brain Weight (in grams)', 'PMI (in hours)',
       'Dx', 'Year of Autopsy', 'Benzodiazepines', 'Anticonvulsants',
       'AntipsychTyp', 'AntipsychAtyp', 'Antidepress', 'Lithium', 'Alcohol',
       'Illicitsub', 'causeOfDeath', 'EV.1', 'EV.2', 'EV.3', 'EV.4', 'EV.5'],
      dtype='object')

The code below performs the following operations:
1. drop variant-level features
1. keep only NeuN_pl samples
1. remove duplicate rows

In [3]:
subjects = calls.drop(calls.columns[:31], axis=1).xs('NeuN_pl', axis=0, level='Tissue').groupby('Individual ID').first() # remove duplicate rows
subjects

,Institution,Reported Gender,Ethnicity,ageOfDeath,Brain Weight (in grams),PMI (in hours),Dx,Year of Autopsy,Benzodiazepines,Anticonvulsants,...,Antidepress,Lithium,Alcohol,Illicitsub,causeOfDeath,EV.1,EV.2,EV.3,EV.4,EV.5
Individual ID,,,,,,,,,,,,,,,,,,,,,
CMC_MSSM_027,MSSM,Male,Caucasian,66.0,1239.0,12.1,SCZ,1994.0,0,0,...,0,0,1,0,1,0.019061,-0.043151,0.005808,0.131348,-0.019458
CMC_MSSM_055,MSSM,Male,Caucasian,66.0,1370.0,23.9,Control,2005.0,0,0,...,0,0,0,0,1,0.020298,0.018617,-0.001543,-0.010232,0.002140
CMC_MSSM_056,MSSM,Female,Caucasian,67.0,1222.0,16.5,Control,2005.0,0,0,...,0,0,0,0,1,0.020853,0.030543,-0.006878,-0.011283,0.009727
CMC_MSSM_069,MSSM,Female,Hispanic,63.0,1214.0,20.2,Control,2004.0,0,0,...,1,0,0,0,1,-0.036995,-0.006866,-0.128398,0.015441,0.062868
CMC_MSSM_097,MSSM,Male,Caucasian,73.0,1036.0,50.6,SCZ,1992.0,0,0,...,0,0,0,0,1,0.020930,0.033332,0.005796,-0.006970,0.003070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CMC_PITT_098,Pitt,Male,Caucasian,56.0,1285.0,14.5,Control,1997.0,0,0,...,0,0,0,0,1,0.021925,0.017359,0.001280,0.002445,0.003736
CMC_PITT_101,Pitt,Male,Caucasian,42.0,1670.0,26.1,Control,1997.0,0,0,...,0,0,0,0,1,0.020204,-0.004967,-0.001258,0.041604,-0.004473
CMC_PITT_113,Pitt,Male,Caucasian,48.0,1575.0,16.6,Control,2000.0,0,0,...,0,0,0,0,1,0.020509,0.025007,-0.001910,-0.031377,0.030459


In [4]:
fpath = '../../results/2020-04-22-upload-to-ndar-from-s3/samples-from-Chaggai.csv'
samples_genewiz = pd.read_csv(fpath)
samples_genewiz = samples_genewiz.rename(dict(zip(samples_genewiz.index, ['CMC_' + x for x in samples_genewiz.CMC_simple_id])), axis=0)
samples_genewiz['Sequencing'] = 'GENEWIZ'
samples_genewiz = samples_genewiz.rename_axis('Individual ID', axis=0)
samples_genewiz

,CMC_simple_id,Dissection,PFC #,Dx,GENEWIZ_serialn,Index,Sequencing
Individual ID,,,,,,,
CMC_PITT_117,PITT_117,PITT,904,SCZ,1,A2,GENEWIZ
CMC_MSSM_364,MSSM_364,MSSM,1141,SCZ,2,A3,GENEWIZ
CMC_MSSM_363,MSSM_363,MSSM,1144,SCZ,3,A4,GENEWIZ
CMC_MSSM_308,MSSM_308,MSSM,1153,SCZ,5,A5,GENEWIZ
CMC_MSSM_339,MSSM_339,MSSM,1155,SCZ,6,A6,GENEWIZ
...,...,...,...,...,...,...,...
CMC_MSSM_379,MSSM_379,MSSM,1238,SCZ,D-MSSM-1238,D4,GENEWIZ
CMC_MSSM_340,MSSM_340,MSSM,1247,SCZ,E-MSSM-1247,E5,GENEWIZ
CMC_MSSM_362,MSSM_362,MSSM,1346,SCZ,F-MSSM-1346,F6,GENEWIZ


In [5]:
subjects = pd.merge(left=subjects, right=samples_genewiz[['Sequencing']], how='left', on='Individual ID')
subjects['Sequencing'] = subjects['Sequencing'].fillna('other')
subjects

,Institution,Reported Gender,Ethnicity,ageOfDeath,Brain Weight (in grams),PMI (in hours),Dx,Year of Autopsy,Benzodiazepines,Anticonvulsants,...,Lithium,Alcohol,Illicitsub,causeOfDeath,EV.1,EV.2,EV.3,EV.4,EV.5,Sequencing
Individual ID,,,,,,,,,,,,,,,,,,,,,
CMC_MSSM_027,MSSM,Male,Caucasian,66.0,1239.0,12.1,SCZ,1994.0,0,0,...,0,1,0,1,0.019061,-0.043151,0.005808,0.131348,-0.019458,GENEWIZ
CMC_MSSM_055,MSSM,Male,Caucasian,66.0,1370.0,23.9,Control,2005.0,0,0,...,0,0,0,1,0.020298,0.018617,-0.001543,-0.010232,0.002140,GENEWIZ
CMC_MSSM_056,MSSM,Female,Caucasian,67.0,1222.0,16.5,Control,2005.0,0,0,...,0,0,0,1,0.020853,0.030543,-0.006878,-0.011283,0.009727,other
CMC_MSSM_069,MSSM,Female,Hispanic,63.0,1214.0,20.2,Control,2004.0,0,0,...,0,0,0,1,-0.036995,-0.006866,-0.128398,0.015441,0.062868,GENEWIZ
CMC_MSSM_097,MSSM,Male,Caucasian,73.0,1036.0,50.6,SCZ,1992.0,0,0,...,0,0,0,1,0.020930,0.033332,0.005796,-0.006970,0.003070,GENEWIZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CMC_PITT_098,Pitt,Male,Caucasian,56.0,1285.0,14.5,Control,1997.0,0,0,...,0,0,0,1,0.021925,0.017359,0.001280,0.002445,0.003736,GENEWIZ
CMC_PITT_101,Pitt,Male,Caucasian,42.0,1670.0,26.1,Control,1997.0,0,0,...,0,0,0,1,0.020204,-0.004967,-0.001258,0.041604,-0.004473,other
CMC_PITT_113,Pitt,Male,Caucasian,48.0,1575.0,16.6,Control,2000.0,0,0,...,0,0,0,1,0.020509,0.025007,-0.001910,-0.031377,0.030459,GENEWIZ


In [6]:
%connect_info

{
  "shell_port": 56181,
  "iopub_port": 41405,
  "stdin_port": 46639,
  "control_port": 36115,
  "hb_port": 38181,
  "ip": "127.0.0.1",
  "key": "c9d366f0-476195e862826d1f144b61d7",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-4ba6ed2b-805a-49b9-a16e-462bef54ccdd.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
